# Make Actual Story Vectors
## 12/7/20
## Make a dict, where for each story there is the wordvec made from the actual stories read 

In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random

from sklearn.decomposition import PCA #for cluster analysis
from gensim.models import KeyedVectors #for word embeddings
import matplotlib.pyplot as plt #for plotting
import os #for importing
import pickle #for loading transcripts
from scipy.stats import pearsonr 

# from _DRAFT_20200604_functions import * #includes constants and score function
from tqdm import tqdm_notebook #for progress bar

%autosave 5

Autosaving every 5 seconds


# I. Imports
## A. Read story texts

In [15]:

files1 = glob.glob('/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/*')
len(files1)

16

In [18]:
for filenam in files1: 
    print(filenam)

/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/11
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/12
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/13
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/14
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/21
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/22
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/23
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/24
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/31
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/32
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/33
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/34
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/stor

In [31]:
files = {}
for filenam in files1: 
    with open(filenam, "r") as file: 
        if filenam in files: 
            continue 
        else:
            x = filenam.split('ns/')[1]
            files[x] = file.read() 

## B. Importing wiki vectors

In [26]:
wikipath = 'rolando/wiki-news-300d-1M.vec'
wv_model = KeyedVectors.load_word2vec_format(wikipath)

# II. Functions

In [27]:
wv_dim = 300

# FastText preprocessing, based on bittlingmayer/ft_wiki_preproc.py
# Remove special characters, put spaces between all tokens
SUB = ["s/’/'/g", "s/′/'/g", "s/''/ /g", "s/'/ ' /g", 's/“/"/g', 's/”/"/g', 's/"/ /g', "s/\\./ \\. /g", "s/<br \\/>/ /g", "s/, / , /g", "s/(/ ( /g", "s/)/ ) /g", "s/\\!/ \\! /g", "s/\\?/ \\? /g", "s/\\;/ /g", "s/\\:/ /g", "s/-/ - /g", "s/=/ /g", "s/=/ /g", "s/*/ /g", "s/|/ /g", "s/«/ /g", 
       "s/…/ /g", "s/‘/ /g", "s/í/ /g", "s/ñ/ /g", "s/\x84/ /g", "s/î/ /g", "s/ó/ /g", "s/\x83/ /g", "s/ï/ /g", "s/õ/ /g",
       "s/ò/ /g", "s/,/ /g", "s/ô/ /g", "s/\x92/ /g", "s/é/ /g", "s/\x8e/ /g", "s/â\x80¦/ /g", "s/\x91/ /g", "s/\x93/ /g",
       "s/\x94/ /g", "s/ã®/ /g", "s/ã¨/ /g", "s/ã©/ /g",
       "s/\â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x9d/ /g", "s/â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x98/ /g",
       "s/â/ /g"]

def __normalize_text(s):
    for sg in SUB:
        rep = sg.replace('\\','').split('/')
        s = s.replace(rep[1], rep[2])
    s = s.replace('/',' ')
    return s

def __spaces(s):
    return ' '.join(s.split())

def __digits(s):
    return ''.join(filter(lambda c: not c.isdigit(), s))

# def preproc(s):
#     return __punctuation(__spaces(__digits(__normalize_text(s.lower()))))

def preproc(s):
    return (__spaces(__digits(__normalize_text(s.lower()))))

def __punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def word2vecSent(sentence, model = 'fasttext'):
    wv_dim = 300 #for glove and fasttext
    
    if model == 'glove':
        wvmodel = glove_model
    elif model == 'fasttext':
        wvmodel = wv_model
        
    words = preproc(sentence).split(' ')
    wv = np.zeros((len(words), wv_dim))
    for i in range(len(words)):
        if words[i] in wvmodel.vocab:
            wv[i,:] = wvmodel.word_vec(words[i])
    
    return wv

## III. Making the actual story vectors
### A. Getting Word Vectors for each free recall 
#### dict{story: story_text}

In [32]:
wv_matrices = {}

# Iterating through each key
for file in files:
    wv_matrices[file] = np.mean(word2vecSent(files[file]), axis = 0)

In [33]:
wv_matrices

{'11': array([-1.24748149e-02,  7.21614830e-03, -1.84389631e-02,  1.58666663e-03,
        -2.50370413e-03, -5.25838517e-02,  1.21317038e-02, -7.99274088e-03,
         7.62222458e-04, -7.21259246e-03,  1.43875554e-02, -5.83199994e-03,
         1.49354075e-02, -6.13037018e-03, -1.57362964e-02, -1.03317035e-02,
         1.00397037e-02,  1.51306666e-02, -1.73440002e-02, -2.76681488e-03,
        -5.46100743e-02, -3.98088895e-03,  1.91703593e-04,  7.06237032e-03,
        -2.94925925e-02, -2.50571853e-02,  1.89448888e-02,  4.65096289e-02,
         1.06026667e-02, -1.90007409e-02,  2.88909629e-02, -1.17997037e-02,
         2.79360000e-02,  4.49022179e-03,  2.40096294e-02,  1.22343704e-02,
        -3.24133329e-03,  2.22136296e-02,  7.18296285e-03,  5.65585184e-03,
        -9.68488882e-03,  1.10063702e-02, -2.10712591e-02,  4.60637045e-03,
         6.52711114e-03, -1.38863705e-02,  1.16192572e-03, -6.79955567e-03,
         4.44237044e-03, -5.16192590e-03, -2.16859243e-03,  2.47792591e-02,
      

### C. Export Word Vector Matrices and sums

In [34]:
pickle.dump( wv_matrices, open( 'actual_story_vectors', "wb" ) )

In [18]:
data_dict['/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b8/10/1_1_online_exp_2020-11-20_12h31.55.742']

,story_presses,A1,A2,A3,A4,A5,A6,A7,A8,distractor1,...,distractor3,distractor4,distractor5,prolific_id,story,p_type,persp,alt_persp,sa_order,recall
0,1,"""8 months""","""compromis""","""yes, frustration""","""cat""","""busy""","""pasta sauce""","""3 empty""","""not sure""","""accoutrements""",...,"""hippocampus""","""brigadeiro""","""muffuletta""",5faa598de2b35a341189d27c,13,2,10,3,0,A couple enters a grocery store. They are prep...
1,1,"""8 months""","""compromis""","""yes, frustration""","""cat""","""busy""","""pasta sauce""","""3 empty""","""not sure""","""accoutrements""",...,"""hippocampus""","""brigadeiro""","""muffuletta""",5faa598de2b35a341189d27c,13,2,10,3,0,A couple enters a grocery store. They are prep...
2,9,"""8 months""","""compromis""","""yes, frustration""","""cat""","""busy""","""pasta sauce""","""3 empty""","""not sure""","""accoutrements""",...,"""hippocampus""","""brigadeiro""","""muffuletta""",5faa598de2b35a341189d27c,13,2,10,3,0,A couple enters a grocery store. They are prep...
3,1,"""8 months""","""compromis""","""yes, frustration""","""cat""","""busy""","""pasta sauce""","""3 empty""","""not sure""","""accoutrements""",...,"""hippocampus""","""brigadeiro""","""muffuletta""",5faa598de2b35a341189d27c,13,2,10,3,0,A couple enters a grocery store. They are prep...
4,1,"""8 months""","""compromis""","""yes, frustration""","""cat""","""busy""","""pasta sauce""","""3 empty""","""not sure""","""accoutrements""",...,"""hippocampus""","""brigadeiro""","""muffuletta""",5faa598de2b35a341189d27c,13,2,10,3,0,A couple enters a grocery store. They are prep...
5,1,"""8 months""","""compromis""","""yes, frustration""","""cat""","""busy""","""pasta sauce""","""3 empty""","""not sure""","""accoutrements""",...,"""hippocampus""","""brigadeiro""","""muffuletta""",5faa598de2b35a341189d27c,13,2,10,3,0,A couple enters a grocery store. They are prep...
6,1,"""8 months""","""compromis""","""yes, frustration""","""cat""","""busy""","""pasta sauce""","""3 empty""","""not sure""","""accoutrements""",...,"""hippocampus""","""brigadeiro""","""muffuletta""",5faa598de2b35a341189d27c,13,2,10,3,0,A couple enters a grocery store. They are prep...
7,9,"""8 months""","""compromis""","""yes, frustration""","""cat""","""busy""","""pasta sauce""","""3 empty""","""not sure""","""accoutrements""",...,"""hippocampus""","""brigadeiro""","""muffuletta""",5faa598de2b35a341189d27c,13,2,10,3,0,A couple enters a grocery store. They are prep...
8,1,"""8 months""","""compromis""","""yes, frustration""","""cat""","""busy""","""pasta sauce""","""3 empty""","""not sure""","""accoutrements""",...,"""hippocampus""","""brigadeiro""","""muffuletta""",5faa598de2b35a341189d27c,13,2,10,3,0,A couple enters a grocery store. They are prep...
9,1,"""8 months""","""compromis""","""yes, frustration""","""cat""","""busy""","""pasta sauce""","""3 empty""","""not sure""","""accoutrements""",...,"""hippocampus""","""brigadeiro""","""muffuletta""",5faa598de2b35a341189d27c,13,2,10,3,0,A couple enters a grocery store. They are prep...
